In [9]:
import pandas as pd
from deep_translator import (GoogleTranslator,
                             PonsTranslator,
                             LingueeTranslator,
                             MyMemoryTranslator,
                             YandexTranslator,
                             single_detection,
                             batch_detection)
import nltk
import string as st
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment import SentimentAnalyzer
import CleanupFunctions as clean

# Haalt info uit csv op.
dfDutch = pd.read_csv("Mock-updata.csv", sep=';')



dfSentiment = pd.DataFrame.from_dict({
    "id":[],
    "text": [],
    "compound": [],
    "neg":[],
    "neu":[],
    "pos":[],
})


stopwords = nltk.corpus.stopwords.words("english")

index = 0

#Loop die er voor zorgt dat alle vertaalde reviews wordt uitgeprint
for col_name, review in dfDutch.iterrows():
    translated = GoogleTranslator(source='dutch', target='english').translate(text=dfDutch.loc[col_name]['Tekst'])
    textlowercase = clean.convert_to_lowercase(translated)
    textclean = clean.punc_clean(textlowercase)
    textstopword = clean.remove_stopword(textclean)
    tokenized = sent_tokenize(textstopword)
    for line in tokenized:     
        sia = SentimentIntensityAnalyzer()
        ss = sia.polarity_scores(line)
        dfTemp = pd.DataFrame.from_dict({
                            "id":[col_name],
                            "text": [tokenized],
                            "compound": [ss["compound"]],
                            "neg":[ss["neg"]],
                            "neu":[ss["neu"]],
                            "pos":[ss["pos"]],
                            })
        index = index + 1
    dfSentiment = pd.concat([dfSentiment, dfTemp], ignore_index=True)


print(dfSentiment)
dfSentiment.to_csv("MockUpDataSentiment.csv")

      id                                               text  compound    neg  \
0    0.0    [totally disagree wait surgery bloody hospital]   -0.6997  0.592   
1    1.0  [receptionist rude makes wait phone waiting 50...   -0.1280  0.122   
2    2.0                            [great sister handsome]    0.8074  0.000   
3    3.0  [satisfied corona booster risk group go back s...    0.8435  0.080   
4    4.0  [unfortunately lung cancer chance surviving si...   -0.7666  0.383   
5    5.0  [broke ribs hurts lot sit emergency room waiti...   -0.7589  0.411   
6    6.0  [give birth hospital teenage mother midwives t...   -0.1779  0.189   
7    7.0  [ive recorded say corona thats total bullshit ...   -0.5938  0.284   
8    8.0  [ive blood tests results arent yet wait dont k...    0.3089  0.000   
9    9.0  [top hospital pay 1006 euros treatment not wor...    0.0000  0.241   
10  10.0  [care poor waiting time simple surgery knee tw...    0.0258  0.233   
11  11.0    [receptionist curt wait long